In [1]:
from CC import CC
from DnsVhost import DnsVhost
from MariaDB import MariaDB
from WP import WP
from Partition import Partition
from SMB import SMB
from NFS import NFS
from Diskquota import Diskquota


## 객체 ###
cc =  CC("172.16.16.122", "root", "asd123!@")
cc.connect()
dv = DnsVhost() 
mariadb = MariaDB()
#wp = WP()
part = Partition()
smb = SMB("Rocky")
nfs = NFS("Rocky")
dq = Diskquota("Rocky")
## 전역변수 ##
ZONE_FILE = ""
IPADDR = ""


###################################################
#              HTTP,DNS -DnsVhost.py              #
###################################################
# 나중에 수정할 필요 있음. DB에 맞춰..
#---------------------HTTP-----------------------#
## 1. http 설치, vhost 수정, 재시작 , DNS설치 및 설정까지 : 밑의 mod_http_R 함수 사용
def install_and_setting_http() :
    cc.run(dv.install_http())
    mod_http_and_dns()

## 2. vhost 수정, 재시작
def mod_http_and_dns():
    cc.run(dv.set_vhostconf())
    set_httpd_and_dns()
    cc.run(dv.restart_http())

## 3. vhost 수정, 만약 zone파일이 존재하면 vhost의 내용을 반영하고 3차도메인을 수정합니다.
def set_httpd_and_dns() :
    vhost_conf = dv.getter_VHOST_CONF()
    print(f"현재 당신의 vhost.conf 파일은 {vhost_conf}입니다.")
    while(True) :
        cc.run(dv.show_vhostconf())
        num = int(input("(1) vhost 추가하기, (0) vhost 파일 수정 종료"))
        if(num == 0) :
            print("vhost 파일 수정을 종료합니다")
            break;
        cc.run(dv.mod_vhostconf())
    cc.run(dv.restart_http())

    print("httpd와 관련한 vhost내용은 성공적으로 반영되었습니다. 이에따른 DNS 내용도 수정하겠습니다")

    ## DNS 내용들임
    install_and_setting_dns()
    #cc.dv.set_dns_Third_domain()

        
#---------------------DNS-----------------------#
## 1. DNS 설치 및 ZONE 파일 수정, vhost에 따른 3차도메인 자동 업데이트
def install_and_setting_dns():
    cc.run(dv.install_named())
    cc.run(dv.set_named_conf())
    cc.run(dv.set_named_rfc())
    cc.run(dv.mk_zone_file())
    
    cc.run(dv.set_zone_file())
    cc.run(dv.set_dns_Third_domain())
    cc.run(dv.start_dns())

#---------------------실행부-----------------------#
#install_and_setting_http()



###################################################
#            MariaDB,PHP -MariaDB.py              #
###################################################
## 1. mariaDB와 php 둘 다 설치함
def install_mariaDB_and_php():
    cc.run(mariadb.install_packages())
    cc.run(mariadb.install_apache_and_php())
    cc.run(mariadb.install_mariaDB())
    cc.run(mariadb.init_mariaDB())
    cc.run(mariadb.install_php_my_admin())
    cc.run(mariadb.set_phpMyadmin_conf())
#---------------------실행부-----------------------#
#install_mariaDB_and_php()


##################################################
#           Partition -Partition.py              #
##################################################
def mk_partition():
    cc.run("lsblk")
    cc.run(part.make_partition())
    cc.run(part.format_partition())
    cc.run(part.mount())
    cc.run(part.permanent_mount_config_remote())


#---------------------실행부-----------------------#
mk_partition()

###################################################
#                    SMB -SMB.py                  #
###################################################

def install_smb():
    cc.run(smb.smb_install())
    cc.run(smb.smb_intsll_status_check())   
    choose = input("smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_smb()
    if(choose == "C" ) :
        client_smb()

def server_smb():
    cc.run(smb.smb_share_folder_make())
    cc.run(smb.smb_mk_smb_id())
    cc.run(smb.smb_set_smb_id_passwd())
    cc.run(smb.smb_id_check())
    cc.run(smb.smb_share_folder_authority())
    cc.run(smb.smb_mk_back())
    cc.run(smb.smb_set())
    cc.run(smb.smb_server_start())
    
def client_smb() :
    cc.run(smb.smb_client_login())
    cc.run(smb.smb_client_mount())
#---------------------실행부-----------------------#
#install_smb()

###################################################
#                    NFS -NFS.py                  #
###################################################

def install_nfs():
    cc.run(nfs.nfs_isntall())
    choose = input("NFS 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_nfs()
    if(choose == "C" ) :
        client_nfs()

def server_nfs():
    cc.run(nfs.nfs_share_folder_make())
    cc.run(nfs.nfs_share_folder_authority())
    cc.run(nfs.server_nfs_set_exports_config())
    cc.run(nfs.server_nfs_start())

def client_nfs():
    cc.run(nfs.nfs_client_mount_temporary())
    cc.run(nfs.nfs_clinet_mount_permanent())
#---------------------실행부-----------------------#
#install_nfs()

###################################################
#            Diskquota -diskquota.py              #
###################################################
def diskquota_install_and_set():
    cc.run(dq.diskquota_isntall())
    cc.run(dq.diskquota_install_check())
    cc.run("lsblk")
    print("TYPE MOUNTPOINTS가 part로 되어있는 디스크만 가능합니다. 안되면 partition을 해야합니다")
    exist_part = input("part로 되어있는 디스크가 없으시거나 해당 디스크가 마운트가 안되어있는 경우에 반드시 n 입력해주세요.\n 있으시면 y 또는 아무문자열 입력 ex) y: ")
    if(exist_part == "n") :
        return
    cc.run(dq.diskquota_modify())
    cc.run(dq.diskquota_mount_and_initalization())
    cc.run(dq.diskquota_settings())
    cc.run(dq.diskquota_systemctl())
    
#---------------------실행부-----------------------#
diskquota_install_and_set()


###################################################
#                 WP,PHPMYSQL -WP.py              #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_wp():
    wp.install_prev_packages()
    wp.install_and_move_dir()
    wp.set_wp_config_p()

실행
DnsVhost init start
MariaDB init start
NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS
sda           8:0    0   10M  0 disk 
sdb           8:16   0   10M  0 disk 
sdc           8:32   0   10M  0 disk 
sdd           8:48   0   10M  0 disk 
sr0          11:0    1  1.8G  0 rom  
nvme0n1     259:0    0   20G  0 disk 
├─nvme0n1p1 259:1    0    1G  0 part /boot
└─nvme0n1p2 259:2    0   19G  0 part 
  ├─rl-root 253:0    0   17G  0 lvm  /
  └─rl-swap 253:1    0    2G  0 lvm  [SWAP]



무엇을 파티션할건지 적으세요 (ex. /dev/sda):  /dev/sda


spawn fdisk /dev/sda

Welcome to fdisk (util-linux 2.37.4).
Changes will remain in memory only, until you decide to write them.
Be careful before using the write command.

Device does not contain a recognized partition table.
Created a new DOS disklabel with disk identifier 0x142ba3d8.

Command (m for help): n
Partition type
   p   primary (0 primary, 0 extended, 4 free)
   e   extended (container for logical partitions)
Select (default p): 

Using default response p.
Partition number (1-4, default 1): 
First sector (2048-20479, default 2048): 
Last sector, +/-sectors or +/-size{K,M,G,T,P} (2048-20479, default 20479): 

Created a new partition 1 of type 'Linux' and of size 9 MiB.

Command (m for help): w
The partition table has been altered.
Calling ioctl() to re-read partition table.
Syncing disks.

NAME        MAJ:MIN RM  SIZE RO TYPE MOUNTPOINTS
sda           8:0    0   10M  0 disk 
└─sda1        8:1    0    9M  0 part 
sdb           8:16   0   10M  0 disk 
sdc           8:32   0   

아까 생성한 파티션을 입력하세요(ex. /dev/sda1):  /dev/sda1


mke2fs 1.46.5 (30-Dec-2021)
Creating filesystem with 9216 1k blocks and 2304 inodes
Filesystem UUID: 731be279-d69b-4737-b936-80963e953f07
Superblock backups stored on blocks: 
	8193

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (1024 blocks): done
Writing superblocks and filesystem accounting information: done

NAME FSTYPE FSVER LABEL UUID                                   FSAVAIL FSUSE% MOUNTPOINTS
sda                                                                           
└─sda1
                                                                              
sdb                                                                           
sdc                                                                           
sdd                                                                           
sr0  iso966 Jolie Rocky-9-5-x86_64-dvd
                        2024-11-16-01-52-31-00                         

...

/dev/sda1 /mnt/test ext4 defaults,usrquota,grpquota 0 0

diskquota 파일을 생성하고 초기화합니다.
tune2fs 1.46.5 (30-Dec-2021)
quotacheck: Cannot find filesystem to check or filesystem not mounted with quota option.
quotaon: using . on /dev/sda1 [/mnt/test]: File exists
quotaon: using . on /dev/sda1 [/mnt/test]: File exists



diskquota 세팅을 하시겠습니까 (y or n) ex)y :  y
설정할 계정명을 입력해주세요 :  test
설정할 그룹명을 입력해주세요 :  test
유예기간을 설정하세요 ex)1 : 1
제한 설정을 선택하세요
                                1. 사용자 제한 설정 
                                2. 그룹 제한 설정
                                3. 유예기간 설정
                                 1


adduser: warning: the home directory /home/test already exists.
adduser: Not copying any file from skel directory into it.



쿼터 기능을 선택하세요 (on/off) : )
                         1. on
                         2. off
                          1


잘못된 입력입니다.

